## Visualize with iPySigma

In [1]:
# Run once cell

%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
import pickle 
import json
import networkx as nx
from ipysigma import Sigma

In [12]:
!jupyter labextension enable ipysigma
!jupyter labextension enable ipysigma user
!jupyter labextension enable ipysigma sys-prefix

### Read Data

In [3]:
df_path = "data/master_dataframe"
edge_path = "data/master_edges"
# df = pd.read_csv(df_path + ".csv")
df = pd.read_pickle(df_path + ".pkl")
edges = pd.read_pickle(edge_path + ".pkl")

In [5]:
df.iloc[0]['pos']

[-8.188774108886719, 106.9360122680664]

### Sigma Graph

In [6]:
# Let's make smaller graphs using only pairs of levels

top_level = 4
bottom_level = 3

df_pair = df[(df['level'] == top_level) | (df['level'] == bottom_level)]
print(df_pair.shape)

# edges_pair = edges[(edges['.isin(df_pair['id'])) & (edges['target'].isin(df_pair['id'])]

(48859, 7)


In [8]:
df_pair.index

Index([   186,    187,    188,    189,    190,    191,    192,    193,    194,
          195,
       ...
       251028, 251029, 251030, 251031, 251032, 251033, 251034, 251035, 251036,
       251037],
      dtype='int64', length=48859)

In [35]:
# Filter edges to include only those that have both source and target in df_pair
edges_pair = {int(k): int(v) for k, v in edges.items() if k in df_pair.index and v in df_pair.index}
len(edges_pair)
# Convert the keys and values of edges_pair to integers


33056

In [38]:
for k, v in edges_pair.items():
    print(k)
    print(type(v))
    break


81100
<class 'int'>


In [11]:
df_pair['level'].unique()

array([3, 4])

In [12]:
df_pair.head()

,pdb_id,level,level_idx,scalar_rep,datum,pos,color
186,1f00I,3,0,"[-0.72847205, -0.20440121, -0.9378011, 0.52105...",(((<moleculib.protein.datum.ProteinDatum objec...,"[29.39433479309082, 79.16131591796875]","rgb(77, 91, 131)"
187,1f00I,3,1,"[-0.3508038, 0.14778979, -0.4626064, -0.218643...",(((<moleculib.protein.datum.ProteinDatum objec...,"[-72.5463638305664, -17.0191707611084]","rgb(160, 221, 139)"
188,1f00I,3,2,"[-0.31163827, 0.32068214, -0.6623976, -0.25478...",(((<moleculib.protein.datum.ProteinDatum objec...,"[14.0927734375, -40.565345764160156]","rgb(91, 170, 118)"
189,1f00I,3,3,"[-0.28070223, 0.25043678, -0.64689237, -0.2482...",(((<moleculib.protein.datum.ProteinDatum objec...,"[-1.4058302640914917, -12.002650260925293]","rgb(98, 149, 104)"
190,1f00I,3,4,"[-0.38578805, 0.27555993, -0.68480414, -0.2867...",(((<moleculib.protein.datum.ProteinDatum objec...,"[-1.7725545167922974, 96.90339660644531]","rgb(150, 203, 205)"


In [36]:
%%time

def nxgraph(df, edges, vertical_shift=250):
    """Make a graph with networkx to send to ipysigma."""
    layout = {
        int(idx): {
            "x": float(row['pos'][0]),
            "y": float(row['pos'][1]) + vertical_shift * (row['level'] - 3),

        } for idx, row in df.iterrows()
    }

    graph = nx.Graph()
    for idx, row in df.iterrows():
        graph.add_node(int(idx), level=int(row['level']), level_idx=int(row['level_idx']))

    graph.add_edges_from(edges.items())
    print(f"There are {graph.number_of_nodes()} nodes in the graph")

    return graph, layout

graph, layout = nxgraph(df_pair, edges_pair, vertical_shift=300)


There are 48859 nodes in the graph
CPU times: user 6 s, sys: 24.1 ms, total: 6.02 s
Wall time: 6.11 s


In [37]:

def make_kwargs(df):

    edge_kwargs = dict(
        default_edge_type="curve",
        default_edge_curveness=0.2,
        default_edge_size=1.0,
        clickable_edges=False,
        hide_edges_on_move=True,
    )

    node_kwargs = dict(
        node_label={int(idx): row['pdb_id'] for idx, row in df.iterrows()},
        raw_node_color=df['color'].values,
        node_border_color_from='node',
    )

    return node_kwargs, edge_kwargs


node_kwargs, edge_kwargs = make_kwargs(df_pair)
PATH_HTML = "lvl3_4.html"
sigma = Sigma.write_html(
    graph,
    path=PATH_HTML,
    layout=layout,
    # node_metrics=['louvain'],
    # node_metrics={"community": {"name": 'louvain', "resolution": 0.5}},
    # node_color='louvain',
    **node_kwargs,
    **edge_kwargs
)

TypeError: Object of type int64 is not JSON serializable